In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
'''
utility definitions and functions
'''
DATA_DIR = '../data/'
PICKLE_DIR = DATA_DIR

In [3]:
COUNTYFP_CODES = {
        '003': 'Albemarle',
        '540': 'Charlottesville',
        '079': 'Greene',
        '109': 'Louisa',
        '125': 'Nelson',
        }

COUNTIES_URL = "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv"

df_county_pop = pd.read_csv(COUNTIES_URL, encoding="latin-1")
df_county_pop['COUNTY'] = df_county_pop['COUNTY'].astype(str).str.zfill(3)
df_county_pop = df_county_pop[(df_county_pop['STNAME'] == 'Virginia') &
                          (df_county_pop['COUNTY'].isin(COUNTYFP_CODES))]
df_county_pop = df_county_pop[['COUNTY', 'POPESTIMATE2019']]
df_county_pop.rename(columns={'COUNTY': 'COUNTYFP'}, inplace=True)

In [4]:
df_county_pop

,COUNTYFP,POPESTIMATE2019
2868,003,109330
2905,079,19819
2920,109,37591
2927,125,14930
2965,540,47266


In [5]:
county_json_url = "https://services.arcgis.com/p5v98VHDX9Atv3l7/arcgis/rest/services/SDE_USDC_CENSUS_VA_COUNTY/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"
gdf_county_shapes = gpd.read_file(county_json_url)

In [6]:
gdf_county_shapes = gdf_county_shapes[gdf_county_shapes['COUNTYFP'].isin(COUNTYFP_CODES)]

In [7]:
gdf_counties=gdf_county_shapes.merge(df_county_pop, how='left', on='COUNTYFP')

first_cols = ['FID','STATEFP','COUNTYFP','COUNTYNS','GEOID','NAME','NAMELSAD','POPESTIMATE2019']
last_cols = [col for col in gdf_counties.columns if col not in first_cols]
gdf_counties = gdf_counties[first_cols+last_cols]

gdf_counties

,FID,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,POPESTIMATE2019,LSAD,CLASSFP,...,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,SHAPE_AREA,SHAPE_LEN,geometry
0,12,51,540,01789068,51540,Charlottesville,Charlottesville city,47266,25,C7,...,16820,,F,26517351,52974,+38.0376579,-078.4853806,4.290251e+07,47968.815482,"POLYGON ((-78.47071 38.04872, -78.47033 38.048..."
1,29,51,125,01500936,51125,Nelson,Nelson County,14930,06,H1,...,16820,,A,1219510173,8934497,+37.7900492,-078.8793535,1.970228e+09,270906.166374,"POLYGON ((-78.96031 37.65626, -78.96048 37.656..."
2,32,51,079,01480129,51079,Greene,Greene County,19819,06,H1,...,16820,,A,404675812,1706931,+38.2979805,-078.4701629,6.608403e+08,145779.950439,"POLYGON ((-78.41995 38.19979, -78.42093 38.200..."
3,95,51,109,01494707,51109,Louisa,Louisa County,37591,06,H1,...,40060,,A,1285411676,37739404,+37.9716808,-077.9591779,2.133052e+09,226038.621899,"POLYGON ((-78.01796 37.89279, -78.01797 37.892..."
4,122,51,003,01675170,51003,Albemarle,Albemarle County,109330,06,H1,...,16820,,A,1866599957,14035664,+38.0241841,-078.5535056,3.035450e+09,293681.034758,"POLYGON ((-78.66084 37.72266, -78.66100 37.722..."


In [8]:
# gdf_counties.to_pickle(path=f'{PICKLE_DIR}counties.pickle')